In [ ]:
!pip install  dgl -f https://data.dgl.ai/wheels/cu116/repo.html
!pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Insert, change the directory 
import sys
sys.path.insert(0,'/content/drive/MyDrive/graph-based-fraud-detection-main')
%cd /content/drive/MyDrive/graph-based-fraud-detection-main

/content/drive/MyDrive/graph-based-fraud-detection-main


In [ ]:
import pandas as pd

In [ ]:
# Read CSV file into a pandas dataframe
df = pd.read_csv('./ieee-data/test_identity.csv')
df1 = pd.read_csv('./ieee-data/test_transaction.csv')

In [ ]:
df.shape, df1.shape

((141907, 41), (506691, 393))

In [ ]:
df1.columns

Index(['TransactionID', 'TransactionDT', 'TransactionAmt', 'ProductCD',
       'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       ...
       'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338',
       'V339'],
      dtype='object', length=393)

In [ ]:
df2 = pd.read_csv('./ieee-data/train_identity.csv')
df3 = pd.read_csv('./ieee-data/train_transaction.csv')

In [ ]:
df2.shape, df3.shape

((144233, 41), (590540, 394))

In [ ]:
# Count the number of unique transaction IDs
count = df['TransactionID'].nunique()
count1 = df1['TransactionID'].nunique()
count2 = df2['TransactionID'].nunique()
count3 = df3['TransactionID'].nunique()

print("Number of unique transaction IDs in file test_identity :", count)
print("Number of unique transaction IDs test_transaction :", count1)
print("Number of unique transaction IDs train_identity :", count2)
print("Number of unique transaction IDs train_transaction :", count3)

Number of unique transaction IDs in file test_identity : 141907
Number of unique transaction IDs test_transaction : 506691
Number of unique transaction IDs train_identity : 144233
Number of unique transaction IDs train_transaction : 590540


In [ ]:
value_counts = df3['isFraud'].value_counts()

# Print the result
print("Number of 1's (is frad):", value_counts[1])
print("Number of 0's (not frad):", value_counts[0])


Number of 1's (is frad): 20663
Number of 0's (not frad): 569877


In [ ]:
print('df2 - train_identity')
print('df3 - train_transaction')

df2 - train_identity
df3 - train_transaction


In [ ]:
train_identity_tids = list(df2['TransactionID'].values)

In [ ]:
train_transaction_tids = list(df3['TransactionID'].values)

In [ ]:
df2[~df2['TransactionID'].isin(train_transaction_tids)]

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo


In [ ]:
df3[~df3['TransactionID'].isin(train_identity_tids)] # transactionids which are not in identity table

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.50,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2987005,0,86510,49.00,W,5937,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.00,W,6550,NaN,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,0,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,0,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,0,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
filter_df = df3[df3['TransactionID'].isin(train_identity_tids)]

In [ ]:
filter_df.shape

(144233, 394)

In [ ]:
value_counts = filter_df['isFraud'].value_counts()

# Print the result
print("Number of 1's (is frad):", value_counts[1])
print("Number of 0's (not frad):", value_counts[0])


Number of 1's (is frad): 11318
Number of 0's (not frad): 132915


In [ ]:
fraud_df = filter_df[filter_df['isFraud']==1].sample(n=1500)
fraud_df.shape

(1500, 394)

In [ ]:
non_fraud_df = filter_df[filter_df['isFraud']==0].sample(n=48500)
non_fraud_df.shape

(48500, 394)

In [ ]:
sample_df = pd.concat([non_fraud_df, fraud_df])

In [ ]:
value_counts = sample_df['isFraud'].value_counts()

# Print the result
print("Number of 1's (is frad):", value_counts[1])
print("Number of 0's (not frad):", value_counts[0])

Number of 1's (is frad): 1500
Number of 0's (not frad): 48500


In [ ]:
sample_df_tids = list(sample_df['TransactionID'].values)

In [ ]:
sample_df_identity = df2[df2['TransactionID'].isin(sample_df_tids)]

In [ ]:
sample_df_identity.shape, sample_df.shape

((50000, 41), (50000, 394))

In [ ]:
# sample_df - train_transaction
# sample_df_identity - train_identity

In [ ]:
sample_df.to_csv('ieee-data/train_transaction.csv', index=False)

In [ ]:
sample_df_identity.to_csv('ieee-data/train_identity.csv', index=False)

In [ ]:
sample_df